#INPUTS

In [ ]:
accountId = '122495099'     # Make sure the email "gaextraction@modern-index-357412.iam.gserviceaccount.com" has "Admin" Level access to the extracted account

#CODE

In [ ]:
import pandas as pd
import json
import os
import time

from googleapiclient.discovery import build
from google.oauth2 import service_account

##GA3

In [ ]:
PATH_TO_FILE = '/content/modern-index-357412-8ad604b5706e.json'
credentials = service_account.Credentials.from_service_account_file(PATH_TO_FILE)

analytics = build('analytics', 'v3', credentials=credentials)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

###Filters

In [ ]:
properties = analytics.management().webproperties().list(accountId=accountId).execute()
filters = analytics.management().filters().list(accountId=accountId).execute()
views = analytics.management().profiles().list(accountId=accountId, webPropertyId='~all').execute()
filters_view_link = analytics.management().profileFilterLinks().list(accountId=accountId, webPropertyId='~all', profileId='~all').execute()
accounts = analytics.management().accounts().list().execute()

df_columns = ['S.No', 'Views Name', 'Filters Detail', 'Account', 'Property Name', 'Property ID', 'Filter Count']
df_dict = {'S.No': "", 'Views Name': "", 'Filters Detail': "", 'Account': "", 'Property Name': "", 'Property ID': "", 'Filter Count': ""}
df = pd.DataFrame(df_columns)

filter_view_dict = {}
view_id_name_dict = {}
property_id_name_dict = {}
account_name = ""

In [ ]:
for filter_view in filters_view_link.get('items', []):
  view_id = filter_view['id'].split(":")[0]
  filter_id = filter_view['id'].split(":")[1]
  if(view_id in filter_view_dict):
    filter_view_dict[view_id].append(filter_id)
  else:
    filter_view_dict[view_id] = [filter_id]

for view in views.get('items', []):
  view_id = view.get('id')
  view_name = view.get('name')
  view_id_name_dict[view_id] = view_name

  filter_id = "None"
  if(view_id not in filter_view_dict):
    filter_view_dict[view_id] = [filter_id]

for account in accounts.get('items', []):
  if(account.get('id') == accountId):
    account_name = account.get('name')

In [ ]:
def getFilter(id):
  if(id == "None"):
    return "No Filters"
  for filter in filters.get('items', []):
    filterId = filter.get('id')
    if(filterId == id):
      filterType = filter.get('type')
      filterName = filter.get('name')
      filterDetails = filterName

      if filterType == 'EXCLUDE':
        details = filter.get('excludeDetails', {})
        filterDetails += 'Exclude field = {0}\n'.format(details.get('field'))
        filterDetails += 'Exclude match type = {0}\n'.format(details.get('matchType'))
        filterDetails += 'Exclude expression value = {0}\n'.format(details.get('expressionValue'))
        filterDetails += 'Exclude case sensitive = {0}\n'.format(details.get('caseSensitive'))

      if filterType == 'INCLUDE':
        details = filter.get('includeDetails', {})
        filterDetails += 'Include field = {0}\n'.format(details.get('field'))
        filterDetails += 'Include match type = {0}\n'.format(details.get('matchType'))
        filterDetails += 'Include expression value = {0}\n'.format(details.get('expressionValue'))
        filterDetails += 'Include case sensitive = {0}\n'.format(details.get('caseSensitive'))

      if filterType == 'LOWERCASE':
        details = filter.get('lowercaseDetails', {})
        filterDetails += 'Lowercase field = {0}\n'.format(details.get('field'))

      if filterType == 'UPPERCASE':
        details = filter.get('uppercaseDetails', {})
        filterDetails += 'Uppercase field = {0}\n'.format(details.get('field'))

      if filterType == 'SEARCH_AND_REPLACE':
        details = filter.get('searchAndReplaceDetails', {})
        filterDetails += 'Search and replace field = {0}\n'.format(details.get('field'))
        filterDetails += 'Search string = {0}\n'.format(details.get('searchString'))
        filterDetails += 'Replace string = {0}\n'.format(details.get('replaceString'))
        filterDetails += 'Search and replace case sensitive = {0}\n'.format(details.get('caseSensitive'))

      if filterType == 'ADVANCED':
        details = filter.get('advancedDetails', {})
        filterDetails += 'Advanced field A = {0}\n'.format(details.get('fieldA'))
        filterDetails += 'Advanced extract A = {0}\n'.format(details.get('extractA'))
        filterDetails += 'Advanced field B = {0}\n'.format(details.get('fieldB'))
        filterDetails += 'Advanced extract B = {0}\n'.format(details.get('extractB'))
        filterDetails += 'Advanced output field = {0}\n'.format(details.get('outputToField'))
        filterDetails += 'Advanced output constructor = {0}\n'.format(details.get('outputConstructor'))
        filterDetails += 'Advanced field A required = {0}\n'.format(details.get('fieldARequired'))
        filterDetails += 'Advanced field B required = {0}\n'.format(details.get('fieldBRequired'))
        filterDetails += 'Advanced override output field = {0}\n'.format(details.get('overrideOutputField'))
        filterDetails += 'Advanced case sensitive = {0}\n'.format(details.get('caseSensitive'))
      return filterDetails

In [ ]:
def getAllViewsOfProperty(id):
  count = 0
  viewDetails = ""
  filterDetails = ""
  details = []
  for view in views.get('items', []):
    propertyId = view.get('webPropertyId')
    if(propertyId == id):
      filterCount = 0
      count += 1
      viewName = view.get('name')
      viewId = view.get('id')

      viewDetails = "{0}: {1}".format(count, viewName)
      filterDetails = ""
      
      filtersIds = filter_view_dict.get(viewId)
      filterCount = len(filtersIds)
      for c, i in enumerate(filtersIds):
        filterDetails += "{0}: {1}\n".format(str(c+1), getFilter(i))
      details.append({'viewDetails': viewDetails, 'filterDetails': filterDetails, 'filterCount': filterCount if filterDetails != "1: No Filters\n" else "0"})

  return details

In [ ]:
count = 0
count1 = 1
for property in properties.get('items', []):
  count += 1
  accountId = '{0}'.format(property.get('accountId'))
  propertyId = '{0}'.format(property.get('id'))
  propertyName = '{0}'.format(property.get('name'))
  details = getAllViewsOfProperty(propertyId)

  df_dict['S.No'] = count
  df_dict['Account'] = account_name
  df_dict['Property Name'] = propertyName
  df_dict['Property ID'] = propertyId

  for i in details:
    df_dict['Views Name'] = i.get('viewDetails')
    df_dict['Filters Detail'] = i.get('filterDetails')
    df_dict['Filter Count'] = i.get('filterCount')

    df_dict = { key:str(value).strip() for key, value in df_dict.items()}

    if(count1 == 1):
      df = pd.DataFrame(df_dict, index=[count])
      count1 += 1
    else:
      df1 = pd.DataFrame(df_dict, index=[count])
      df = df.append([df1])
      
    df_dict['Views Name'] = None
    df_dict['Filter Count'] = None
    df_dict['Filters Detail'] = None

  df_dict = {'S.No': "", 'Views Name': "", 'Filters Detail': "", 'Account': "", 'Property Name': "", 'Property ID': "", 'Filter Count': ""}

###CDs, CMs and Audience

In [ ]:
df_cm_columns = ['S.No', 'ID', 'Name', 'Index', 'Active', 'Scope', 'Type', 'Property']
df_cm_dict = {'S.No': "", 'ID': "", 'Name': "", 'Index': "", 'Active': "", 'Scope': "", 'Type': "", 'Property': ""}

df_cd_columns = ['S.No', 'ID', 'Name', 'Index', 'Active', 'Scope', 'Property']
df_cd_dict = {'S.No': "", 'ID': "", 'Name': "", 'Index': "", 'Active': "", 'Scope': "", 'Property': ""}

df_audiences_columns = ['S.No', 'Id', 'Name', 'Description', 'Property', 'Type', 'Linked Views', 'Linked AdAccounts', 'Definition']
df_audiences_dict = {'S.No': "", 'Id': "", 'Name': "", 'Description': "", 'Property': "", 'Type': "", 'Linked Views': "", 'Linked AdAccounts': "", 'Definition': ""}

df_cm = pd.DataFrame(columns = df_cm_columns)
df_cd = pd.DataFrame(columns = df_cd_columns)
df_audiences = pd.DataFrame(columns = df_audiences_columns)

In [ ]:
dimensions = []
metrics = []
audiences = []
count_cd = 1
count_cm = 1
count_audience = 1

for i in properties.get('items', []):
  property_id_name_dict[i['id']] = i['name']
  dimensions = analytics.management().customDimensions().list(accountId=accountId,webPropertyId=i['id']).execute()['items']
  metrics = analytics.management().customMetrics().list(accountId=accountId,webPropertyId=i['id']).execute()['items']
  audiences = analytics.management().remarketingAudience().list(accountId=accountId,webPropertyId=i['id']).execute()['items']

  if(dimensions):
    count = 1
    for dimension in dimensions:
      df_cd_dict['S.No'] = count
      df_cd_dict['ID'] = dimension.get('id')
      df_cd_dict['Name'] = dimension.get('name')
      df_cd_dict['Index'] = dimension.get('index')
      df_cd_dict['Active'] = dimension.get('active')
      df_cd_dict['Scope'] = dimension.get('scope')
      df_cd_dict['Property'] = dimension.get('webPropertyId')

      count += 1
      df_cd_dict = { key:str(value).strip() for key, value in df_cd_dict.items()}

      if(count_cd == 1):
        df_cd = pd.DataFrame(df_cd_dict, index=[count_cd])
      else:
        df1 = pd.DataFrame(df_cd_dict, index=[count_cd])
        df_cd = df_cd.append([df1])

      count_cd += 1
      df_cd_dict = {'S.No': "", 'ID': "", 'Name': "", 'Index': "", 'Active': "", 'Scope': "", 'Property': ""}

    for i in range(5):
      df_cd = df_cd.append(pd.DataFrame(df_cd_dict, index=[0]))

  if(metrics):
    count = 1
    for metric in metrics:
      df_cm_dict['S.No'] = count
      df_cm_dict['ID'] = metric.get('id')
      df_cm_dict['Name'] = metric.get('name')
      df_cm_dict['Index'] = metric.get('index')
      df_cm_dict['Active'] = metric.get('active')
      df_cm_dict['Scope'] = metric.get('scope')
      df_cm_dict['Type'] = metric.get('type')
      df_cm_dict['Property'] = metric.get('webPropertyId')

      count += 1
      df_cm_dict = { key:str(value).strip() for key, value in df_cm_dict.items()}

      if(count_cm == 1):
        df_cm = pd.DataFrame(df_cm_dict, index=[count_cm])
      else:
        df1 = pd.DataFrame(df_cm_dict, index=[count_cm])
        df_cm = df_cm.append([df1])
      
      count_cm += 1
      df_cm_dict = {'S.No': "", 'ID': "", 'Name': "", 'Index': "", 'Active': "", 'Scope': "", 'Type': "", 'Property': ""}  

    for i in range(5):
      df_cm = df_cm.append(pd.DataFrame(df_cm_dict, index=[0]))
  
  if(audiences):
    count = 1
    for audience in audiences:
      count_audience += 1
      df_audiences_dict['S.No'] = count
      df_audiences_dict['Id'] = audience.get('id')
      df_audiences_dict['Name'] = audience.get('name')
      df_audiences_dict['Description'] = audience.get('description')
      df_audiences_dict['Property'] = audience.get('webPropertyId')

      linkedViews = ""
      for view in audience.get('linkedViews'):
        linkedViews += '{0}\n'.format(view)

      df_audiences_dict['Linked Views'] = linkedViews

      linkedAdAccounts = ""
      for link in audience.get('linkedAdAccounts', []):
        linkedAdAccounts += 'Link type = {0}\n'.format(link.get('type'))
        linkedAdAccounts += 'Property = {0}\n'.format(link.get('webPropertyId'))
        linkedAdAccounts += 'Status = {0}\n'.format(link.get('status'))
        linkedAdAccounts += 'Linked AccountId = {0}\n\n'.format(link.get('linkedAccountId'))

      df_audiences_dict['Linked AdAccounts'] = linkedAdAccounts

      audienceType = audience.get('audienceType')
      df_audiences_dict['Type'] = audienceType

      audienceDefinition = ""
      if audienceType == 'SIMPLE':
        definition = audience.get('audienceDefinition', {})
        # Get the include conditions.
        condition = definition.get('includeConditions', {})
        audienceDefinition += 'daysToLookBack = {0}\n'.format(condition.get('daysToLookBack'))
        audienceDefinition += 'membershipDurationDays = {0}\n'.format(condition.get('membershipDurationDays'))
        audienceDefinition += 'segment = {0}\n'.format(condition.get('segment'))
      elif audienceType == 'STATE_BASED':
        definition = audience.get('stateBasedAudienceDefinition', {})
        # get the include conditions
        condition = definition.get('includeConditions', {})
        audienceDefinition += 'daysToLookBack = {0}\n'.format(condition.get('daysToLookBack'))
        audienceDefinition += 'membershipDurationDays = {0}\n'.format(condition.get('membershipDurationDays'))
        audienceDefinition += 'segment = {0}\n'.format(condition.get('segment'))
        # get the exclude condition
        condition = definition.get('excludeConditions', {})
        audienceDefinition += 'exclusionDuration = {0}\n'.format(condition.get('exclusionDuration'))
        audienceDefinition += 'segment = {0}\n'.format(condition.get('segment'))

      df_audiences_dict['Definition'] = audienceDefinition

      count += 1
      df_audiences_dict = { key:str(value).strip() for key, value in df_audiences_dict.items()}

      if(count_audience == 1):
        df_audiences = pd.DataFrame(df_audiences_dict, index=[count_audience])
      else:
        df1 = pd.DataFrame(df_audiences_dict, index=[count_audience])
        df_audiences = df_audiences.append([df1])

      count_audience += 1
      df_audiences_dict = {'S.No': "", 'Id': "", 'Name': "", 'Description': "", 'Property': "", 'Type': "", 'Linked Views': "", 'Linked AdAccounts': "", 'Definition': ""}

    for i in range(5):
      df_audiences = df_audiences.append(pd.DataFrame(df_audiences_dict, index=[0]))

  time.sleep(0.1)  # so that we dont exceed API Quota limit per second 

###Property and View Settings

In [ ]:
df_property_columns = ['S.No', 'Account Name', 'Property Name', 'Property ID', 'Host Name', 'Property Level', 'Industry Vertical', 'DataRetention Reset On New Activity', 'DataRetention TTL', 'View Count']
df_property_dict = {'S.No': "", 'Account Name': "", 'Property Name': "", 'Property ID': "", 'Host Name': "", 'Property Level': "", 'Industry Vertical': "", 'DataRetention Reset On New Activity': "", 'DataRetention TTL': "", 'View Count': ""}

df_view_columns = ['S.No', 'View Name', 'View ID', 'Host Name', 'Property Name', 'Property ID', 'Bot Filtering', 'eCommerce Tracking', 'Excluded Query Parameters', 'Timezone', 'SiteSearch Query Parameters', 'Starred', 'Type']
df_view_dict = {'S.No': "", 'View Name': "", 'View ID': "", 'Host Name': "", 'Property Name': "", 'Property ID': "", 'Bot Filtering': "", 'eCommerce Tracking': "", 'Excluded Query Parameters': "", 'Timezone': "", 'SiteSearch Query Parameters': "", 'Starred': "", 'Type': ""}

df_property = pd.DataFrame(df_property_columns)
df_view = pd.DataFrame(df_view_columns)

In [ ]:
count = 0
for property in properties.get('items', []):
  count += 1
  accountId = '{0}'.format(property.get('accountId'))
  propertyId = '{0}'.format(property.get('id'))
  propertyName = '{0}'.format(property.get('name'))
  hostName = '{0}'.format(property.get('websiteUrl'))
  propertyViewCount = '{0}'.format(property.get('profileCount'))  
  propertyLevel = '{0}'.format(property.get('level'))
  industryVertical = '{0}'.format(property.get('industryVertical'))
  dataRetentionResetOnNewActivity = '{0}'.format(property.get('dataRetentionResetOnNewActivity'))
  dataRetentionTtl = '{0}'.format(property.get('dataRetentionTtl'))

  df_property_dict['S.No'] = count
  df_property_dict['Property Level'] = "GA 360" if(propertyLevel == "PREMIUM") else "GA"
  df_property_dict['Account Name'] = account_name
  df_property_dict['Property Name'] = propertyName
  df_property_dict['Property ID'] = propertyId
  df_property_dict['View Count'] = propertyViewCount
  df_property_dict['Host Name'] = hostName
  df_property_dict['Industry Vertical'] = industryVertical
  df_property_dict['DataRetention Reset On New Activity'] = dataRetentionResetOnNewActivity
  df_property_dict['DataRetention TTL'] = dataRetentionTtl

  if(count == 1):
    df_property = pd.DataFrame(df_property_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_property_dict, index=[count])
    df_property = df_property.append([df1])
  
  df_property_dict = {'S.No': "", 'Account Name': "", 'Property Name': "", 'Property ID': "", 'Host Name': "", 'Property Level': "", 'Industry Vertical': "", 'DataRetention Reset On New Activity': "", 'DataRetention TTL': "", 'View Count': ""}
  

count = 0
for view in views.get('items', []):
  count += 1
  name = '{0}'.format(view.get('name'))
  id = '{0}'.format(view.get('id'))
  propertyId = '{0}'.format(view.get('webPropertyId'))
  propertyName = '{0}'.format(property_id_name_dict[propertyId])
  hostName = '{0}'.format(view.get('websiteUrl'))
  botFilteringEnabled = '{0}'.format(view.get('botFilteringEnabled'))
  eCommerceTracking = '{0}'.format(view.get('eCommerceTracking'))
  timezone = '{0}'.format(view.get('timezone'))
  type_ = '{0}'.format(view.get('type'))
  excludeQueryParameters = '{0}'.format(view.get('excludeQueryParameters'))
  siteSearchQueryParameters = '{0}'.format(view.get('siteSearchQueryParameters'))
  starred = '{0}'.format(view.get('starred'))

  df_view_dict['S.No'] = count
  df_view_dict['View Name'] = name
  df_view_dict['View ID'] = id
  df_view_dict['Host Name'] = hostName
  df_view_dict['Property Name'] = propertyName
  df_view_dict['Property ID'] = propertyId
  df_view_dict['Bot Filtering'] = "Enabled" if(botFilteringEnabled == "True") else "Disabled"
  df_view_dict['eCommerce Tracking'] = eCommerceTracking
  df_view_dict['Timezone'] = timezone
  df_view_dict['Starred'] = starred
  df_view_dict['Excluded Query Parameters'] = excludeQueryParameters
  df_view_dict['SiteSearch Query Parameters'] = siteSearchQueryParameters
  df_view_dict['Type'] = type_

  if(count == 1):
    df_view = pd.DataFrame(df_view_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_view_dict, index=[count])
    df_view = df_view.append([df1])
  
  df_view_dict = {'S.No': "", 'View Name': "", 'View ID': "", 'Host Name': "", 'Property Name': "", 'Property ID': "", 'Bot Filtering': "", 'eCommerce Tracking': "", 'Excluded Query Parameters': "", 'Timezone': "", 'SiteSearch Query Parameters': "", 'Starred': "", 'Type': ""}

###Goals

In [ ]:
df_goals_columns = ['S.No', 'Number', 'Name', 'Value', 'Type', 'Active', 'Details', 'Property ID', 'View ID', 'Property Name', 'View Name']
df_goals_dict = {'S.No': "", 'Number': "", 'Name': "", 'Value': "", 'Type': "", 'Active': "", 'Details': "", 'Property ID': "", 'View ID': "", 'Property Name': "", 'View Name': ""}

df_goals = pd.DataFrame(df_columns)

In [ ]:
def print_url_destination_goal_details(goal_details):
  goalDetails = "Goal URL = {0}\n".format(goal_details.get('url'))
  goalDetails += "Case Sensitive = {0}\n".format(goal_details.get('caseSensitive'))
  goalDetails += "Match Type = {0}\n".format(goal_details.get('matchType'))
  goalDetails += "First Step Required = {0}\n".format(goal_details.get('firstStepRequired'))

  for goal_step in goal_details.get('steps', []):
    goalDetails += "\nStep Number = {0}\n".format(goal_step.get('number'))
    goalDetails += "Step Name = {0}\n".format(goal_step.get('name'))
    goalDetails += "Step URL = {0}\n".format(goal_step.get('url'))

  return goalDetails

def print_visit_time_on_site_goal_details(goal_details):
  goalDetails = "Comparison Type = {0}\n".format(goal_details.get('comparisonType'))
  goalDetails += "comparison Value = {0}\n".format(goal_details.get('comparisonValue'))

  return goalDetails

def print_visit_num_pages_goal_details(goal_details):
  goalDetails = "Comparison Type = {0}\n".format(goal_details.get('comparisonType'))
  goalDetails += "comparison Value = {0}\n".format(goal_details.get('comparisonValue'))

  return goalDetails

def print_event_goal_details(goal_details):
  goalDetails = "Use Event Value = {0}\n".format(goal_details.get('useEventValue'))

  for event_condition in goal_details.get('eventConditions', []):
    event_type = event_condition.get('type')
    goalDetails += "Type = {0}\n".format(event_type)

    if event_type == 'VALUE':
      goalDetails += "Comparison Type = {0}\n".format(event_condition.get('comparisonType'))
      goalDetails += "Comparison Value = {0}\n\n".format(event_condition.get('comparisonValue'))

    else:  # CATEGORY, ACTION, LABEL types.
      goalDetails += "Match Type = {0}\n".format(event_condition.get('matchType'))
      goalDetails += "Expression = {0}\n\n".format(event_condition.get('expression'))

  return goalDetails

In [ ]:
goals = analytics.management().goals().list(accountId=accountId,webPropertyId='~all',profileId='~all').execute()
count = 0
for goal in goals.get('items', []):
  count += 1
  df_goals_dict['S.No'] = count
  df_goals_dict['Property ID'] = goal.get('webPropertyId')
  df_goals_dict['View ID'] = goal.get('profileId')
  df_goals_dict['Property Name'] = property_id_name_dict[goal.get('webPropertyId')]
  df_goals_dict['View Name'] = view_id_name_dict[goal.get('profileId')]
  df_goals_dict['Number'] = goal.get('id')
  df_goals_dict['Name'] = goal.get('name')
  df_goals_dict['Value'] = goal.get('value')
  df_goals_dict['Active'] = goal.get('active')
  df_goals_dict['Type'] = goal.get('type')

  if goal.get('urlDestinationDetails'):
    df_goals_dict['Details'] = print_url_destination_goal_details(goal.get('urlDestinationDetails'))

  elif goal.get('visitTimeOnSiteDetails'):
    df_goals_dict['Details'] = print_visit_time_on_site_goal_details(goal.get('visitTimeOnSiteDetails'))

  elif goal.get('visitNumPagesDetails'):
    df_goals_dict['Details'] = print_visit_num_pages_goal_details(goal.get('visitNumPagesDetails'))

  elif goal.get('eventDetails'):
    df_goals_dict['Details'] = print_event_goal_details(goal.get('eventDetails'))

  if(count == 1):
    df_goals = pd.DataFrame(df_goals_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_goals_dict, index=[count])
    df_goals = df_goals.append([df1])

###Access Management

In [ ]:
df_account_user_links_columns = ['S.No', 'User Email', 'Permissions effective', 'Permissions local', 'Account Id', 'Account Name']
df_account_user_links_dict = {'S.No': "", 'User Email': "", 'Permissions effective': "", 'Permissions local': "", 'Account Id': "", 'Account Name': ""}

df_property_user_links_columns = ['S.No', 'User Email', 'Permissions effective', 'Permissions local', 'Property Id', 'Property Name']
df_property_user_links_dict = {'S.No': "", 'User Email': "", 'Permissions effective': "", 'Permissions local': "", 'Property Id': "", 'Property Name': ""}

df_view_user_links_columns = ['S.No', 'User Email', 'Permissions effective', 'Permissions local', 'View Id', 'View Name']
df_view_user_links_dict = {'S.No': "", 'User Email': "", 'Permissions effective': "", 'Permissions local': "", 'View Id': "", 'View Name': ""}

df_account_user_links = pd.DataFrame(columns = df_account_user_links_columns)
df_property_user_links = pd.DataFrame(columns = df_property_user_links_columns)
df_view_user_links = pd.DataFrame(columns = df_view_user_links_columns)

In [ ]:
account_user_links = analytics.management().accountUserLinks().list(accountId=accountId).execute()
property_user_links = analytics.management().webpropertyUserLinks().list(accountId=accountId,webPropertyId='~all').execute()
view_user_links = analytics.management().profileUserLinks().list(accountId=accountId,webPropertyId='~all',profileId='~all').execute()

def printList(items):
  res = ""
  if(items):
    for i in items:
      res += "{0} \n".format(i)
  return res

count = 0
for accountUserLink in account_user_links.get('items', []):
  count += 1
  entity = accountUserLink.get('entity', {})
  accountRef = entity.get('accountRef', {})
  userRef = accountUserLink.get('userRef', {})
  permissions = accountUserLink.get('permissions', {})

  df_account_user_links_dict['S.No'] = count
  df_account_user_links_dict['User Email'] = userRef.get('email')
  df_account_user_links_dict['Permissions effective'] = printList(permissions.get('effective'))
  df_account_user_links_dict['Permissions local'] = printList(permissions.get('local'))
  df_account_user_links_dict['Account Id'] = accountRef.get('id')
  df_account_user_links_dict['Account Name'] = accountRef.get('name')

  if(count == 1):
    df_account_user_links = pd.DataFrame(df_account_user_links_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_account_user_links_dict, index=[count])
    df_account_user_links = df_account_user_links.append([df1])


count = 0
for propertyUserLink in property_user_links.get('items', []):
  count += 1
  entity = propertyUserLink.get('entity', {})
  propertyRef = entity.get('webPropertyRef', {})
  userRef = propertyUserLink.get('userRef', {})
  permissions = propertyUserLink.get('permissions', {})

  df_property_user_links_dict['S.No'] = count
  df_property_user_links_dict['User Email'] = userRef.get('email')
  df_property_user_links_dict['Permissions effective'] = printList(permissions.get('effective'))
  df_property_user_links_dict['Permissions local'] = printList(permissions.get('local'))
  df_property_user_links_dict['Property Id'] = propertyRef.get('id')
  df_property_user_links_dict['Property Name'] = propertyRef.get('name')

  if(count == 1):
    df_property_user_links = pd.DataFrame(df_property_user_links_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_property_user_links_dict, index=[count])
    df_property_user_links = df_property_user_links.append([df1])

count = 0
for profileUserLink in view_user_links.get('items', []):
  count += 1
  entity = profileUserLink.get('entity', {})
  profileRef = entity.get('profileRef', {})
  userRef = profileUserLink.get('userRef', {})
  permissions = profileUserLink.get('permissions', {})

  df_view_user_links_dict['S.No'] = count
  df_view_user_links_dict['User Email'] = userRef.get('email')
  df_view_user_links_dict['Permissions effective'] = printList(permissions.get('effective'))
  df_view_user_links_dict['Permissions local'] = printList(permissions.get('local'))
  df_view_user_links_dict['View Id'] = profileRef.get('id')
  df_view_user_links_dict['View Name'] = profileRef.get('name')

  if(count == 1):
    df_view_user_links = pd.DataFrame(df_view_user_links_dict, index=[count])
  else:
    df1 = pd.DataFrame(df_view_user_links_dict, index=[count])
    df_view_user_links = df_view_user_links.append([df1])

#Output

In [ ]:
df.set_index('S.No')
df_property.set_index('S.No')
df_view.set_index('S.No')
df_cd.set_index('S.No')
df_cm.set_index('S.No')
df_goals.set_index('S.No')
df_audiences.set_index('S.No')
df_account_user_links.set_index('S.No')
df_property_user_links.set_index('S.No')
df_view_user_links.set_index('S.No')

writer = pd.ExcelWriter("{0} GA Summary.xlsx".format(account_name))

df_property.to_excel(writer, 'Property', engine='xlswriter', index=False)
df_view.to_excel(writer, 'View', engine='xlswriter', index=False)
df_cd.to_excel(writer, 'Custom Dimentions', engine='xlswriter', index=False)
df_cm.to_excel(writer, 'Custom Metrics', engine='xlswriter', index=False)
df.to_excel(writer, 'Filters', engine='xlswriter', index=False)
df_goals.to_excel(writer, 'Goals', engine='xlswriter', index=False)
df_audiences.to_excel(writer, 'Audiences', engine='xlswriter', index=False)
df_account_user_links.to_excel(writer, 'Account User Management', engine='xlswriter', index=False)
df_property_user_links.to_excel(writer, 'Property User Management', engine='xlswriter', index=False)
df_view_user_links.to_excel(writer, 'View User Management', engine='xlswriter', index=False)

writer.save()

In [ ]:
analytics.close()